# Improved GPT

このファイルではGPT from scratchで学習したモデルの改善を目標とする。このファイルの立ち位置は同リポジトリに含まれている
- pytorch_command.ipynb
- attention_from_scratch.ipynb
- GPT_from_scratch.ipynb<br>
の次に読むことを想定されている。

In [8]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
import warnings
warnings.simplefilter('ignore')
print("CUDA環境が壊れていないことを祈りながら確認-> ", torch.cuda.is_available())

CUDA環境が壊れていないことを祈りながら確認-> True


### PreLN
GPT from scratchでは、学習中に勾配が変化しない状態がいつまで経っても続く時があった。<br>
これはLayer Normalization層がMultiheadAttentionやFFN層の後に置かれるというPostLNの形をとっているからであると最近では言われている<br>
このため、改善点の一つとして、サブレイヤーの前にLayer Normalization層を置くPreLNを行う。<br>
最近のGPTモデルではPreLNを行なっているようである。

In [2]:
class PreLNGPTDecoderLayer(nn.Module):
    def __init__(self, embedding_dim, ffn_dim, num_heads, drop_out_rate = 0., layer_eps=1e-05, batch_first = False):
        super().__init__()
        self.maskedmultiheadattention = nn.MultiheadAttention(embedding_dim, num_heads,batch_first=batch_first)
        self.dropout_selfattn = nn.Dropout(p = drop_out_rate)
        self.layernorm_selfattn = nn.LayerNorm(embedding_dim, eps = layer_eps)

        self.ffn = nn.Sequential(nn.Linear(embedding_dim, ffn_dim), nn.GELU(), nn.Linear(ffn_dim, embedding_dim))#GELUに変更
        self.layernorm_ffn = nn.LayerNorm(embedding_dim, eps = layer_eps)
        self.dropout_ffn = nn.Dropout(p = drop_out_rate)

    def forward(self, x, pad_mask_self = None, mask_self=None):
        #PreLNにする
        dx = self.layernorm_selfattn(x)

        dx, _ = self.maskedmultiheadattention(dx,dx,dx,key_padding_mask = pad_mask_self, attn_mask = mask_self)

        dx = self.dropout_selfattn(dx)

        x = x+dx

        dx = self.layernorm_ffn(x)

        dx = self.dropout_ffn(self.ffn(dx))

        x = x + dx
        return x

### 改善したGPT

このPreLNGPTDecoderLayerを用いてGPTモデルを制作する。ところで、今回はGPTのgenerate_sentence関数でもtemperatureとtopKという手法を追加した。<br>
これにより多様性のある文章が生成される。

In [119]:
class GPT(nn.Module):
    def __init__(self, vocab_size, embedding_dim, ffn_dim, num_heads, drop_out_rate = 0.,\
                  layer_eps=1e-05, batch_first = False, T = 10000, N = 1):
        super().__init__()
        #Tはmax_lenを表している
        self.embedding = nn.Embedding(vocab_size, embedding_dim,)
        self.positional_embedding = nn.Embedding(T, embedding_dim)
        self.decoder = nn.ModuleList([PreLNGPTDecoderLayer(embedding_dim, ffn_dim, num_heads, drop_out_rate,\
                                                               layer_eps, batch_first) for _ in range(N)])
        self.linear = nn.Linear(embedding_dim, vocab_size, bias = False)
        self.vocab_size = vocab_size
    def forward(self, x, y = None,pad_mask_self = None, mask_self=None):
        """
        yはxを1つだけずらしたデータである
        x = data[a:b]なら、y = data[a+1:b+1]となる。
        """
        x = self.embedding(x)
        pos = torch.arange(0,x.size(1),dtype=torch.long).unsqueeze(0).to(x.device)
        pos = self.positional_embedding(pos)
        x = x + pos
        for layer in self.decoder:
            x = layer(x, pad_mask_self = pad_mask_self, mask_self = mask_self)
        x = self.linear(x)
        if y != None:
            eps = 0.1
            loss = F.cross_entropy(x.view(-1, x.size(-1)), y.view(-1), ignore_index=-1, label_smoothing= eps / (self.vocab_size - 1)) 
            #ignore_index=-1はyをonehotベクトル化しないでcross_entropyを使うために使用
            pred = x.argmax(dim = -1).detach().cpu()
            return torch.clamp(loss, min = 1e-32, max = 10e9),pred
        loss = None
        pred = x[:,[-1],:]
        return loss, pred
    def create_mask(self, x: torch.tensor, x_pad: int, device: str):
        """
        (batch_size, sequence_length, embedding_dim)の入力を想定
        """
        """
        Trueが無視される値であることに注意すること
        """
        seq_len = x.size(1)
        #srcのマスク制作
        padding_mask = (x == x_pad)
        mask = torch.triu(torch.ones(size = (seq_len, seq_len))==1).transpose(0,1) #下三角行列を作る
        mask = mask.float().masked_fill(mask == 0, float("-inf")).masked_fill(mask==1.,float(0.0)).to(device)
        return padding_mask, mask

    @torch.no_grad()
    def generate(self,bos: str, sentence_size, tokenizer, device):
        self.eval()
        bos_tokenized = tokenizer.encode_ordinary(bos)
        bos_tokenized = bos_tokenized[-sentence_size:]
        bos_tokenized = torch.LongTensor([bos_tokenized])
        _, add_sentence = self(bos_tokenized.to(device))
        self.train()
        return add_sentence
    
    @torch.no_grad()
    def generate_sentence(self, bos: str, sentence_size, generate_tokens, tokenizer, device, top_K = None, temperature = 1.0):
        return_sentence = bos
        for i in range(generate_tokens):
            add_sentence = self.generate(return_sentence, sentence_size, tokenizer,device)
            add_sentence = add_sentence[:,-1,:] / temperature #(1, vocab_size)
            if top_K is not None:
                v, _ = torch.topk(add_sentence, min(top_K, add_sentence.size(-1)))
                #v[:, [-1]]がtopkの中でも最小値を取る。これより小さいやつは予想に含めない。
                add_sentence[add_sentence < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(add_sentence, dim = -1)
            idx_next = torch.multinomial(probs, num_samples=1)
            return_sentence += tokenizer.decode_batch(idx_next.tolist())[0]
        return return_sentence